In [22]:
class ESIOS:
    def __init__(self):
        #define API parameters
        pass
    
    def get_training_data(start_date, end_date, inputs):
        '''
        Download data from ESIOS API considering dates and specified demand.
        '''

        urls = {
            "demand": f"https://api.esios.ree.es/indicators/1775?type=3&start_date={start_date}&end_date={end_date}",
            "wind": f"https://api.esios.ree.es/indicators/1777?type=20&start_date={start_date}&end_date={end_date}",
            "solar": f"https://api.esios.ree.es/indicators/1779?type=3&start_date={start_date}&end_date={end_date}",
            "spot_price": f"https://api.esios.ree.es/indicators/602?type=3&start_date={start_date}&end_date={end_date}"
        }
        try:

            url = urls[inputs]

            # headers for the API
            headers = dict()
            headers['Accept'] = 'application/json; application/vnd.esios-api-v1+json'
            headers['Content-Type'] = 'application/json'
            headers['Host'] = 'api.esios.ree.es'
            headers['x-api-key'] = "27ac7b794ca773e7d1c6ea0f43adfd466abe7dbd6682b769ddd29cf25536c1d6"
            headers['Cookie'] = ''

            # connect to the API
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                data = response.json()
                # Process the data as needed
            else:
                print("Error: Request failed with status code", response.status_code)

            return data

        except Exception as e:

            print("could not retrieve data")
            print(e)
    

    def extract_values(data, call_data):
        '''
        Extracts values and time from the ESIOS data and returns a Pandas DataFrame.
        '''
        #values = [v["value"] for v in data["indicator"]["values"]]
        #times = [v["datetime"] for v in data["indicator"]["values"]]
        try:
            values = data["indicator"]["values"]
            df = pd.DataFrame(values)
            df = df[['value','datetime']]
            
            return df
        
        except Exception as e:
            print(f'No values could be extracted for {call_data}')
            print(e)
        # Create a DataFrame with 'time' and 'call_data' columns
        #df = pd.DataFrame({ 'time': times, call_data: values })

    def create_indicators_file ():

        if os.path.isfile("filename.txt"):
            print('The indicators file is already created!')


        else:
            try:    
                indicators = "https://api.esios.ree.es/indicators"
                headers = dict()
                headers['Accept'] = 'application/json; application/vnd.esios-api-v1+json'
                headers['Content-Type'] = 'application/json'
                headers['Host'] = 'api.esios.ree.es'
                headers['x-api-key'] = "27ac7b794ca773e7d1c6ea0f43adfd466abe7dbd6682b769ddd29cf25536c1d6"
                headers['Cookie'] = ''

                # connect to the API
                response = requests.get(indicators, headers=headers)

                if response.status_code == 200:
                    data = response.json()
                    # Process the data as needed
                else:
                    print("Error: Request failed with status code", response.status_code)

                df = pd.DataFrame(data["indicators"])
                df.to_csv('indicators.csv')
                print('Indicators file created succesfully!')
            except Exception as e:
                print('Indicators file could not be created')
                print(e)

    def connection_to_DB(df, call_data, year):
        '''
        Calls marginal_to_df and loads the df into a SQLite db
        '''

        try:


            conn = sqlite3.connect('Main_DB')
            c = conn.cursor()
            '''
            # Creates table to store marginalpdbc
            c.execute(f''''''CREATE TABLE IF NOT EXISTS {call_data} (
                    year number, 
                    month number, 
                    day number, 
                    hour number, 
                    price_ES number, 
                    price_PT number,
                    PRIMARY KEY (year, month, day, hour)
                    )
                    '''''')
            conn.commit()
            '''
            # Send dataframe to db
            df.to_sql(f'{call_data}', conn, if_exists='append', index=False,
                    method='multi', chunksize=1000)
            '''
            # Check for duplicates in the
            c.execute(f''''''
            DELETE FROM {call_data}
            WHERE ROWID NOT IN (
            SELECT MIN(ROWID) FROM {call_data}
            GROUP BY year, month, day, hour)
            '''''')
            '''
            print(f'{call_data} data uploaded succesfully for year {year}')



        except Exception as e:
            print("Could not upload the data to the DB")
            print(e)

In [23]:
start_date = "2019-01-01T00:00:00"

In [24]:
import pandas as pd

# Sample DataFrame with a column containing date-time values
data = {'DateTimeColumn': ['2019-01-01T00:00:00.000+01:00', '2020-02-02T12:30:00.000+02:00']}
df = pd.DataFrame(data)

# Splitting the DateTimeColumn into Date and Time components
df[['Date', 'Time']] = df['DateTimeColumn'].str.split('T', expand=True)

# Display the modified DataFrame
print(df)


                  DateTimeColumn        Date                Time
0  2019-01-01T00:00:00.000+01:00  2019-01-01  00:00:00.000+01:00
1  2020-02-02T12:30:00.000+02:00  2020-02-02  12:30:00.000+02:00


In [28]:
df['Time'] = df['Time'].str[:5]

In [29]:
df

,DateTimeColumn,Date,Time,Hour
0,2019-01-01T00:00:00.000+01:00,2019-01-01,00:00,00:00
1,2020-02-02T12:30:00.000+02:00,2020-02-02,12:30,12:30


In [20]:
gas_data['Delivery day'] = gas_data['Delivery day'].dt.strftime('%Y-%m-%d')

In [21]:
gas_data

,Delivery day,MIBGAS PVB Last Price Index Day-Ahead\n[EUR/MWh],MIBGAS PVB Average Price Index Day-Ahead\n[EUR/MWh],MIBGAS VTP Last Price Index Day-Ahead\n[EUR/MWh],MIBGAS VTP Average Price Index Day-Ahead\n[EUR/MWh],MIBGAS-ES Index\n[EUR/MWh],MIBGAS-PT Index\n[EUR/MWh],MIBGAS\nLNG-ES Index\n[EUR/MWh],MIBGAS\nAVB-ES Index\n[EUR/MWh]
0,2023-12-19,33.94,32.73,34.18,NaN,NaN,NaN,NaN,NaN
1,2023-12-18,31.06,30.90,31.23,NaN,33.53,32.50,31.37,31.88
2,2023-12-17,30.68,30.74,30.83,NaN,32.54,32.50,34.40,31.88
3,2023-12-16,30.68,30.74,30.83,NaN,31.92,32.50,30.80,31.88
4,2023-12-15,31.78,32.31,32.06,NaN,32.09,32.50,31.78,31.88
...,...,...,...,...,...,...,...,...,...
348,2023-01-05,58.00,57.45,NaN,NaN,58.74,65.00,56.60,70.00
349,2023-01-04,63.00,67.29,NaN,70.0,64.91,70.00,62.41,70.00
350,2023-01-03,60.70,57.73,NaN,NaN,67.00,73.00,62.43,67.00
351,2023-01-02,54.00,57.94,NaN,NaN,62.71,59.67,56.00,64.93


In [6]:
import sqlite3
import pandas as pd

# Connect to the SQLite database (if the database doesn't exist, it will be created)
conn = sqlite3.connect('Main_DB')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Your SQL query
create_values_df = '''
SELECT * from gas_data g
JOIN demand d ON d.day = g.delivery_day
'''

# Read SQL query result into a DataFrame
df = pd.read_sql_query(create_values_df, conn)

# Close the connection
conn.close()

# Display the DataFrame
df

,delivery_day,index_price,value,day,hour
0,2019-12-31,11.80,19914.0,2019-12-31,04:00
1,2019-12-31,11.80,20196.0,2019-12-31,03:00
2,2019-12-31,11.80,20236.0,2019-12-31,05:00
3,2019-12-31,11.80,21021.0,2019-12-31,02:00
4,2019-12-31,11.80,21574.0,2019-12-31,06:00
...,...,...,...,...,...
178789,2022-01-01,99.00,23862.0,2022-01-01,19:00
178790,2022-01-01,99.00,24014.0,2022-01-01,22:00
178791,2022-01-01,99.00,24550.0,2022-01-01,20:00
178792,2022-01-01,99.00,24947.0,2022-01-01,21:00


In [2]:
import sqlite3
import pandas as pd

# Connect to the SQLite database (if the database doesn't exist, it will be created)
conn = sqlite3.connect('Main_DB')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Your SQL query
create_values_df = '''
SELECT * from solar
'''

# Read SQL query result into a DataFrame
solar = pd.read_sql_query(create_values_df, conn)

# Close the connection
conn.close()

# Display the DataFrame
solar

,value,day,hour
0,0.0,2019-01-01,00:00
1,0.0,2019-01-01,01:00
2,0.0,2019-01-01,02:00
3,0.0,2019-01-01,03:00
4,0.0,2019-01-01,04:00
...,...,...,...
35063,0.0,2022-12-31,20:00
35064,0.0,2022-12-31,21:00
35065,0.0,2022-12-31,22:00
35066,0.0,2022-12-31,23:00


In [69]:
import sqlite3
import pandas as pd

# Connect to the SQLite database (if the database doesn't exist, it will be created)
conn = sqlite3.connect('Main_DB')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Your SQL query
temp_df = '''
SELECT * from wind
'''

# Read SQL query result into a DataFrame
wind = pd.read_sql_query(temp_df, conn)
wind.rename(columns={'value': 'wind_generation'}, inplace=True)

# Your SQL query
temp_df = '''
SELECT * from spot_price
'''

# Read SQL query result into a DataFrame
spot_price = pd.read_sql_query(temp_df, conn)

# Read SQL query result into a DataFrame
spot_price = pd.read_sql_query(temp_df, conn)
spot_price.rename(columns={'value': 'spot_price'}, inplace=True)

# Your SQL query
temp_df = '''
SELECT * from solar
'''

# Read SQL query result into a DataFrame
solar = pd.read_sql_query(temp_df, conn)
solar.rename(columns={'value': 'solar_generation'}, inplace=True)

# Your SQL query
temp_df = '''
SELECT * from demand
'''

# Read SQL query result into a DataFrame
demand = pd.read_sql_query(temp_df, conn)
demand.rename(columns={'value': 'demand'}, inplace=True)

# SQL query
temp_df = '''
SELECT * from gas_data
'''

# Read SQL query result into a DataFrame
gas_data = pd.read_sql_query(temp_df, conn)
gas_data.rename(columns={'value': 'gas_price'}, inplace=True)
gas_data_hourly = gas_data.reindex(gas_data.index.repeat(24)).reset_index(drop=True)

# Close the connection
conn.close()

# Display the DataFrame


In [71]:
df=wind.merge(spot_price, left_on=['day', 'hour'], right_on=['day', 'hour']) \
        .merge(solar, left_on=['day', 'hour'], right_on=['day', 'hour']) \
        .merge(demand, left_on=['day', 'hour'], right_on=['day', 'hour']) \
        .merge(gas_data, left_on=['day'], right_on=['delivery_day'])\
        .drop('delivery_day', axis=1)

df

,wind_generation,day,hour,spot_price,solar_generation,demand,index_price
0,3214.0,2019-01-01,00:00,66.88,0.0,23753.0,24.45
1,3222.0,2019-01-01,01:00,66.88,0.0,23018.0,24.45
2,3081.0,2019-01-01,02:00,66.00,0.0,21808.0,24.45
3,3069.0,2019-01-01,03:00,63.64,0.0,20635.0,24.45
4,2973.0,2019-01-01,04:00,58.85,0.0,19824.0,24.45
...,...,...,...,...,...,...,...
50948,10402.5,2022-12-31,22:00,1.88,0.0,21761.5,57.91
50949,10402.5,2022-12-31,22:00,1.88,0.0,21761.5,59.67
50950,10305.0,2022-12-31,23:00,0.10,0.0,20590.3,57.91
50951,10305.0,2022-12-31,23:00,0.10,0.0,20590.3,59.67


In [73]:
gas_data_hourly = gas_data.reindex(gas_data.index.repeat(24)).reset_index(drop=True)
gas_data_hourly

,delivery_day,index_price
0,2019-12-31,11.80
1,2019-12-31,11.80
2,2019-12-31,11.80
3,2019-12-31,11.80
4,2019-12-31,11.80
...,...,...
59347,2023-01-01,59.42
59348,2023-01-01,59.42
59349,2023-01-01,59.42
59350,2023-01-01,59.42


In [83]:
import requests
import pandas as pd
import sqlite3
import openpyxl
import io

def get_data(year):
        
        url = f'https://www.mibgas.es/es/file-access/MIBGAS_Data_{year}.xlsx?path=AGNO_{year}/XLS'

        if year == 2023:
            workbook_sheet = 'MIBGAS Indexes'
        else:
            workbook_sheet = 'Indices'

        # GET request to the URL
        response = requests.get(url)

        if response.status_code == 200:
            # Load the content of the XLSX file into a BytesIO buffer
            file_content = io.BytesIO(response.content)
            
            # Read the XLSX file using openpyxl
            workbook = openpyxl.load_workbook(file_content)
            
            # Get the names of all sheets in the workbook
            indexes = workbook[workbook_sheet]

            data = indexes.values
            
            data_list = list(data)
            df = pd.DataFrame(data_list[1:], columns=data_list[0])
            
            # drop nan columns by date
            df = df.dropna(subset=['Delivery day'])
            
            return df  # Return the list of sheet names
        else:
            print('Failed to retrieve the webpage. Status code:', response.status_code)
            return None
        
test = get_data(2022)
test

,Delivery day,Area,MIBGAS Index\n[EUR/MWh],MIBGAS Volume\n[MWh],MIBGAS\nLNG Index\n[EUR/MWh],MIBGAS\nLNG Volume\n[MWh],MIBGAS\nAVB Index\n[EUR/MWh],MIBGAS\nAVB Volume\n[MWh]
0,2022-12-31,ES,57.91,265020.0,56.00,300.0,73.64,0.0
1,2022-12-31,PT,59.67,0.0,NaN,0.0,NaN,0.0
2,2022-12-30,ES,62.68,275773.0,64.54,3260.0,73.64,0.0
3,2022-12-30,PT,59.67,0.0,NaN,0.0,NaN,0.0
4,2022-12-29,ES,63.05,314062.0,67.45,1138.0,73.64,1030.0
...,...,...,...,...,...,...,...,...
725,2022-01-03,PT,99.00,0.0,NaN,0.0,NaN,0.0
726,2022-01-02,ES,59.79,177894.0,70.94,0.0,30.50,0.0
727,2022-01-02,PT,99.00,0.0,NaN,0.0,NaN,0.0
728,2022-01-01,ES,57.89,196432.0,70.94,160.0,30.50,0.0


In [82]:
test = test.loc[test['Area']=='ES']
test

KeyError: 'Area'